# Interactive Optimisation Notebook

Use the sliders to tune parameters and run the optimisation.

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# path placeholders
bank_master_path = 'bank_master.csv'
fee_table_path = 'fee_table.csv'
balance_path = 'balance_snapshot.csv'
cashflow_path = 'cashflow_history.csv'

def load_data():
    df_bank = pd.read_csv(bank_master_path)
    df_fee = pd.read_csv(fee_table_path)
    df_bal = pd.read_csv(balance_path)
    df_cash = pd.read_csv(cashflow_path)
    return df_bank, df_fee, df_bal, df_cash


In [ ]:
def calc_safety(df_cash, horizon_days=30, quantile=0.95):
    df = df_cash.copy()
    df['date'] = pd.to_datetime(df['date'])
    df['net'] = df.apply(lambda r: r.amount if r.direction=='out' else -r.amount, axis=1)
    roll = (
        df.set_index('date').groupby('bank_id')['net']
        .rolling(f"{horizon_days}D").sum().reset_index()
    )
    q = roll.groupby('bank_id')['net'].quantile(quantile)
    return q.round().astype(int)


In [ ]:
def optimise_plan(df_bank, df_fee, df_bal, df_cash, lam):
    # placeholder optimisation - return empty dataframe
    return pd.DataFrame(columns=['execute_date', 'from_bank', 'to_bank', 'service_id', 'amount', 'expected_fee'])


In [ ]:
h_slider = widgets.IntSlider(value=30, min=7, max=60, description="horizon")
q_slider = widgets.FloatSlider(value=0.95, min=0.5, max=0.99, step=0.01, description="quantile")
lambda_slider = widgets.FloatSlider(value=1.0, min=0, max=5, step=0.1, description="lambda")
run_button = widgets.Button(description="Run optimisation")
out = widgets.Output()

def on_run_clicked(b):
    out.clear_output()
    df_bank, df_fee, df_bal, df_cash = load_data()
    safety = calc_safety(df_cash, h_slider.value, q_slider.value)
    plan = optimise_plan(df_bank, df_fee, df_bal, df_cash, lambda_slider.value)
    with out:
        display(safety)
        display(plan)
        plan.to_csv("transfer_plan.csv", index=False)
        print("Saved transfer_plan.csv")
run_button.on_click(on_run_clicked)
display(h_slider, q_slider, lambda_slider, run_button, out)
